In [ ]:
# imports from resemblyzer demo01
from resemblyzer import preprocess_wav, VoiceEncoder
from itertools import groupby
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.axes_grid1 import make_axes_locatable

# my imports
import os



/home/blake/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/blake/.local/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [14]:
sample_dir = '../audio/samples/'

artist_dirs = {name: sample_dir + name + '/' for name in 
        ['justin_bieber',
         'kanye_west',
         'lil_uzi_vert',
         'post_malone']}

print(artist_dirs)

# all of the wav file paths (starting from root dir)
wav_paths = {artist:paths for (artist,paths) in 
             [(a,
               
               sorted(list(map(lambda x: artist_dirs[a] + x, os.listdir(artist_dirs[a]))),key=lambda x: float(x.split("_")[-1][:-4]))
              
              ) for a in artist_dirs]}

#print(wav_paths)

{'justin_bieber': '../audio/samples/justin_bieber/', 'kanye_west': '../audio/samples/kanye_west/', 'lil_uzi_vert': '../audio/samples/lil_uzi_vert/', 'post_malone': '../audio/samples/post_malone/'}


In [15]:
encoder = VoiceEncoder()

Loaded the voice encoder model on cpu in 0.06 seconds.


In [16]:
# Dict of the form {artist1: {filename1: wav1, filename2: wav2 ...}, artist2: {filename1: wav1, filename2: wav2 ...} ...
wavs = {artist:dict(wavs) for (artist, wavs) in 
        [(a, list(map(lambda x: (x,preprocess_wav(x)), wav_paths[a]))) for a in wav_paths]}
#print(wavs)

In [21]:
# An embed is a vector of 256 values that summarizes the characteristics of the voice spoken
# (from https://github.com/resemble-ai/Resemblyzer)

# Dict of the form {artist1: {file1: embed1, file2: embed2 ...}, artist2: {file1: embed1, file2: embed2 ...} ...}
embeds = {artist:embeds for (artist, embeds) in 
         [(a, {f:np.array(encoder.embed_utterance(wav))
               for f,wav in wavs[a].items()}) for a in wavs]}

In [1]:
artists_to_compare = ('justin_bieber', 'post_malone')

artist1 = artists_to_compare[0]
artist2 = artists_to_compare[1]

# Compute and show a similarity matrix
#print(embeds)
matrix = np.array([[np.dot(embed1, embed2) for embed1 in 
                    
                    list(map(lambda x: x[1] if ("0.0" in x[0] or "0.0" in f) else np.array([0 for n in range(256)]), embeds[artist1].items()))
                   
                   ][::-1] for f,embed2 in embeds[artist2].items()])

print(matrix)

get_name = lambda p : "/".join(p.split("/")[-2:]) 

x_labels = list(map(get_name,embeds[artist1].keys()))
y_labels = list(map(get_name, embeds[artist2].keys()))

_, ax = plt.subplots()
fig = plt.gcf()
img = ax.matshow(matrix, extent=(-0.5, matrix.shape[0] - 0.5, 
                                     -0.5, matrix.shape[1] - 0.5))

ax.xaxis.set_ticks_position("bottom")
ax.set_xticks(range(len(x_labels)))
ax.set_xticklabels(x_labels[::-1], rotation=90)

ax.set_yticks(range(len(y_labels)))
ax.set_yticklabels(y_labels[::-1])

ax.set_title("Does {0} sound like {1}?".format(artist1, artist2))

cax = make_axes_locatable(ax).append_axes("right", size="5%", pad=0.15)
fig.colorbar(img, cax=cax, ticks=np.linspace(0.4, 1, 7))
img.set_clim(0.4, 1)
img.set_cmap("inferno")
    
plt.show()

plt.close()

NameError: name 'np' is not defined

In [310]:
np.dot([1,2], [3,4])

11

In [314]:
"0.0" in "hello_0.0.wav"

True